In [1]:
# pip install  --upgrade fastapi uvicorn requests ibm-watsonx-orchestrate fastapi_app

In [2]:
from dotenv import load_dotenv
import os
from typing import Optional


load_dotenv()

True

In [3]:
IBM_APIKEY: str = os.getenv("IBM_APIKEY")
ORCHESTRATE_URL: str = os.getenv("ORCHESTRATE_URL")
MODEL: str = os.getenv("MODEL")
REDIS_HOST: str = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT: int = int(os.getenv("REDIS_PORT", "6379"))
REDIS_USERNAME: Optional[str] = os.getenv("REDIS_USERNAME")
REDIS_PASSWORD: Optional[str] = os.getenv("REDIS_PASSWORD")
REDIS_DB: int = int(os.getenv("REDIS_DB", "0"))

In [5]:
!orchestrate env activate watsonx-orchestrate-challenge --api-key={IBM_APIKEY}

[INFO] - Environment 'watsonx-orchestrate-challenge' is now active


In [6]:
!orchestrate agents list

                                     Agents                                     
┏━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓
┃         ┃         ┃         ┃         ┃         ┃         ┃ Knowle… ┃        ┃
┃ Name    ┃ Descri… ┃ LLM     ┃ Style   ┃ Collab… ┃ Tools   ┃ Base    ┃ ID     ┃
┡━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩
│ Assessm │ Presen… │ watsonx │ default │         │ get_ne… │         │ 3c621e │
│ ent_Age │ English │ /openai │         │         │ submit… │         │ fd-84b │
│ nt      │ assess… │ /gpt-os │         │         │         │         │ 8-46ea │
│         │ one     │ s-120b  │         │         │         │         │ -9e0a- │
│         │ questi… │         │         │         │         │         │ 42c67b │
│         │ at a    │         │         │         │         │         │ 0ba5ba │
│         │ time    │         │         │         │         │         │        │
│         │ with    │       

In [7]:
### Agent-ID - 1b22ba03-dcfb-4d0d-8905-a4b081e840aa
# Service instance URL https://api.us-south.watson-orchestrate.cloud.ibm.com/instances/94e10eac-3d20-4334-a133-bbcef5f36c41

In [8]:
import requests

apikey = IBM_APIKEY
url = "https://iam.cloud.ibm.com/identity/token"

headers = { "Content-Type": "application/x-www-form-urlencoded" }
data = {
    "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
    "apikey": apikey
}

response = requests.post(url, headers=headers, data=data)
token = response.json()["access_token"]
# print("Bearer Token:", token)


In [9]:
import http.client
import json

conn = http.client.HTTPSConnection("api.us-south.watson-orchestrate.cloud.ibm.com")

payload = json.dumps(
    {
        "model": "watsonx/meta-llama/llama-3-1-70b-instruct",
        "messages": [
            {
              "role": "system","content": ("Provide a single, complete response in one message. Do NOT ask the user any questions." 
              "If information seems missing or unclear, make reasonable assumptions and continue. Never request clarification. Respond with a concise but complete answer.")},            
            {"role": "user", "content": "What are Nvdia stocks today. Compare them with Microsoft and HDFC Bank. "}
        ],
        "stream": False,
    }
)

headers = {
    "Authorization": f"Bearer {token}",
    "IAM-API_KEY": IBM_APIKEY,
    "content-type": "application/json",
    "accept": "application/json",
}

conn.request(
    "POST",
    "/instances/94e10eac-3d20-4334-a133-bbcef5f36c41/v1/orchestrate/1b22ba03-dcfb-4d0d-8905-a4b081e840aa/chat/completions",
    payload,
    headers,
)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"id":"19404eb4-da28-4396-9322-0cdb49387827","object":"chat.completion","created":1756974779,"model":"watsonx/meta-llama/llama-3-2-90b-vision-instruct","choices":[{"index":0,"message":{"role":"assistant","content":" The current stock prices for NVIDIA, Microsoft, and HDFC Bank are $170.62, $505.35, and $70.62 respectively."},"finish_reason":"stop"}],"thread_id":"09df9330-a742-4461-8ee0-3f81d93b08a4"}


In [10]:
# import requests

# apikey = IBM_APIKEY
# url = "https://iam.cloud.ibm.com/identity/token"

# headers = {"Content-Type": "application/x-www-form-urlencoded"}
# data = {
#     "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
#     "apikey": apikey
# }

# res = requests.post(url, headers=headers, data=data)
# token = res.json()["access_token"]
# print("Bearer token:", token)


In [11]:
import requests
import json

# ---- Step 1: Get IAM token ----
apikey = IBM_APIKEY
iam_url = "https://iam.cloud.ibm.com/identity/token"

iam_headers = {"Content-Type": "application/x-www-form-urlencoded"}
iam_data = {
    "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
    "apikey": apikey
}

iam_res = requests.post(iam_url, headers=iam_headers, data=iam_data)
token = iam_res.json()["access_token"]

# ---- Step 2: Call Watson Orchestrate API ----
api_url = ("https://api.us-south.watson-orchestrate.cloud.ibm.com/"
           "instances/94e10eac-3d20-4334-a133-bbcef5f36c41/v1/"
           "orchestrate/1b22ba03-dcfb-4d0d-8905-a4b081e840aa/chat/completions")

payload = {
    "model": "watsonx/meta-llama/llama-3-1-70b-instruct",
    "messages": [
        {
            "role": "system",
            "content": (
                "Provide a single, complete response in one message. "
                "Do NOT ask the user any questions. "
                "If information seems missing or unclear, make reasonable assumptions and continue. "
                "Never request clarification. Respond with a concise but complete answer."
            )
        },
        {
            "role": "user",
            "content": "What are Nvidia stocks today? Compare them with Microsoft and HDFC Bank."
        }
    ],
    "stream": False
}

api_headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json",
    "Accept": "application/json"
}

api_res = requests.post(api_url, headers=api_headers, json=payload)

print(api_res.status_code)
print(json.dumps(api_res.json(), indent=2))


200
{
  "id": "ae0e37ad-0d6d-492e-87dc-a2fdf3e1f9a2",
  "object": "chat.completion",
  "created": 1756974791,
  "model": "watsonx/meta-llama/llama-3-2-90b-vision-instruct",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": " Nvidia stock price is $170.62, Microsoft stock price is $505.35, and HDFC Bank stock price is $70.62."
      },
      "finish_reason": "stop"
    }
  ],
  "thread_id": "6b317dd1-c5b8-44bb-a725-d68aa102de8d"
}


In [12]:
api_res.text

'{"id":"ae0e37ad-0d6d-492e-87dc-a2fdf3e1f9a2","object":"chat.completion","created":1756974791,"model":"watsonx/meta-llama/llama-3-2-90b-vision-instruct","choices":[{"index":0,"message":{"role":"assistant","content":" Nvidia stock price is $170.62, Microsoft stock price is $505.35, and HDFC Bank stock price is $70.62."},"finish_reason":"stop"}],"thread_id":"6b317dd1-c5b8-44bb-a725-d68aa102de8d"}'

In [13]:
# from fastapi import FastAPI, HTTPException
# from pydantic import BaseModel
# import requests

# app = FastAPI()

# # --- Replace with your own values ---
# IBM_APIKEY = IBM_APIKEY
# ORCHESTRATE_URL = (
#     "https://api.us-south.watson-orchestrate.cloud.ibm.com/"
#     "instances/94e10eac-3d20-4334-a133-bbcef5f36c41/v1/"
#     "orchestrate/1b22ba03-dcfb-4d0d-8905-a4b081e840aa/chat/completions"
# )
# MODEL = "watsonx/meta-llama/llama-3-1-70b-instruct"


# class Prompt(BaseModel):
#     query: str


# def get_iam_token():
#     token_url = "https://iam.cloud.ibm.com/identity/token"
#     headers = {"Content-Type": "application/x-www-form-urlencoded"}
#     data = {
#         "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
#         "apikey": IBM_APIKEY,
#     }
#     res = requests.post(token_url, headers=headers, data=data)
#     if res.status_code != 200:
#         raise HTTPException(status_code=500, detail="Failed to get IAM token")
#     return res.json()["access_token"]


# def call_orchestrate(token: str, query: str):
#     headers = {
#         "Authorization": f"Bearer {token}",
#         "Content-Type": "application/json",
#         "Accept": "application/json",
#     }
#     payload = {
#         "model": MODEL,
#         "messages": [
#             {
#                 "role": "system",
#                 "content": (
#                     "Provide a single, complete response in one message. "
#                     "Do NOT ask the user any questions. "
#                     "If information seems missing or unclear, make reasonable assumptions and continue. "
#                     "Never request clarification. Respond with a concise but complete answer."
#                 ),
#             },
#             {"role": "user", "content": query},
#         ],
#         "stream": False,
#     }

#     res = requests.post(ORCHESTRATE_URL, headers=headers, json=payload)
#     if res.status_code != 200:
#         raise HTTPException(status_code=res.status_code, detail=res.text)
#     data = res.json()
#     return data["choices"][0]["message"]["content"]


# @app.post("/ask")
# def ask(prompt: Prompt):
#     token = get_iam_token()
#     answer = call_orchestrate(token, prompt.query)
#     return {"response": answer}


In [14]:
# !uvicorn fastapi_app:app --reload --host 0.0.0.0 --port 8000

In [15]:
%%bash

curl -X POST http://127.0.0.1:8000/ask \
     -H "Content-Type: application/json" \
     -d '{"query": "What are Nvidia stocks today? Compare them with Microsoft and HDFC Bank."}'


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   309  100   224  100    85      8      3  0:00:28  0:00:26  0:00:02    53100    85      0      5  0:00:17  0:00:16  0:00:01     0   0


{"response":" Nvidia's current stock price is $170.62. Comparing it with Microsoft, the current stock price of Microsoft is $505.35. For HDFC Bank, the information is not available in the search results.","status":"success"}

In [16]:
import requests

url = "http://127.0.0.1:8000/ask"
payload = {"query": "who is CEO of IBM"}

response = requests.post(url, json=payload)
print(response.text)


{"response":" Arvind Krishna","status":"success"}


In [17]:
response = requests.post("https://siri-orchestrate3.1z5ceh9pbx8o.us-south.codeengine.appdomain.cloud/ask", json=payload)
print(response.json())

{'response': ' Arvind Krishna', 'status': 'success'}


In [18]:
response.text

'{"response":" Arvind Krishna","status":"success"}'

In [19]:
# import requests
# import concurrent.futures
# import random

# url = "http://127.0.0.1:8000/ask"

# # Example stock queries
# companies = [
#     "Nvidia", "Microsoft", "Citi Bank", "Google", "HDFC Bank", "Apple", 
#     "Tesla", "Meta", "Amazon", "Intel", "AMD", "IBM", "JP Morgan", "Goldman Sachs"
# ]

# def generate_payload(i):
#     # Randomly pick 4 companies to compare
#     picks = random.sample(companies, 4)
#     query = f"What are {picks[0]} stocks today? Compare them with {picks[1]}, {picks[2]}, and {picks[3]}."
#     return {"query": query, "id": i}

# def send_request(i):
#     payload = generate_payload(i)
#     try:
#         response = requests.post(url, json=payload, timeout=180)  # <-- updated timeout
#         return (i, response.status_code, payload["query"][:80])  # limit output
#     except Exception as e:
#         return (i, "error", str(e))

# # Launch 300 concurrent POST requests
# with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
#     futures = [executor.submit(send_request, i) for i in range(300)]
#     for future in concurrent.futures.as_completed(futures):
#         print(future.result())


In [20]:
"""Basic connection example.
"""

import redis

r = redis.Redis(
    host='redis-18246.c280.us-central1-2.gce.redns.redis-cloud.com',
    port=18246,
    decode_responses=True,
    username="default",
    password=REDIS_PASSWORD,
)

success = r.set('foo', 'bar')
# True

result = r.get('foo')
print(result)
# >>> bar



bar


In [ ]:
import redis
import json

r = redis.Redis(
    host='redis-18246.c280.us-central1-2.gce.redns.redis-cloud.com',
    port=18246,
    decode_responses=True,
    username="default",
    password=REDIS_PASSWORD,
)

session_id = "chat:12345"  # use a unique session key

def add_message(role, content):
    message = {"role": role, "content": content}
    # push to the left, keep list trimmed to last 50 messages (optional)
    r.lpush(session_id, json.dumps(message))
    r.ltrim(session_id, 0, 49)  # keep max 50 messages in history

def get_last_messages(n=5):
    # get latest n messages (from left → index 0 is most recent)
    raw_msgs = r.lrange(session_id, 0, n-1)
    # reverse to get chronological order (oldest first)
    return [json.loads(msg) for msg in reversed(raw_msgs)]

# --- Usage ---
add_message("user", "Hello, what’s the weather today?")
add_message("assistant", "It's sunny and 75°F.")
add_message("user", "How about tomorrow?")

last_msgs = get_last_messages(5)
for m in last_msgs:
    print(f"{m['role']}: {m['content']}")


assistant: It's sunny and 75°F.
user: How about tomorrow?
user: Hello, what’s the weather today?
assistant: It's sunny and 75°F.
user: How about tomorrow?


In [22]:
last_msgs = get_last_messages(5)
for m in last_msgs:
    print(f"{m['role']}: {m['content']}")

assistant: It's sunny and 75°F.
user: How about tomorrow?
user: Hello, what’s the weather today?
assistant: It's sunny and 75°F.
user: How about tomorrow?


In [23]:
import redis

r = redis.Redis(
    host='redis-18246.c280.us-central1-2.gce.redns.redis-cloud.com',
    port=18246,
    decode_responses=True,
    username="default",
    password=REDIS_PASSWORD,
)

# Insert latest message
# r.lpush("messages", )

# Retrieve last 5 messages
last_five = r.lrange("messages", 0, 4)

print(last_five)


[' Arvind Krishna', ' Arvind Krishna', " Nvidia's current stock price is $170.62. Comparing it with Microsoft, the current stock price of Microsoft is $505.35. For HDFC Bank, the information is not available in the search results.", ' The current weather in Mumbai is not explicitly stated in the search results provided.', ' The addition of sine 9 and cos 15 is -0.3475694276170647.']
